In [2]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [0]:
# Access a specific split (e.g., "train")
train_dataset = dataset["train"]
test_dataset = dataset["test"]

# Shuffle and select 2000 samples from the train split
sampled_train_dataset = train_dataset.shuffle(seed=42).select(range(20000))
sampled_test_dataset = test_dataset.shuffle(seed=42).select(range(1000))

In [0]:
# Base model definition
base_model_id = "google-bert/bert-base-cased" #"Qwen/Qwen2.5-0.5B-Instruct" # 

In [0]:
# Loading the Base model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

base_model_id = "google-bert/bert-base-cased" #"Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

# Set the padding token to be the same as eos token (if the model does not have a padding token)
tokenizer.pad_token = tokenizer.eos_token

# Load the model and make sure padding token is set in its configuration
model = AutoModelForSequenceClassification.from_pretrained(base_model_id, num_labels=5)

# Set the model configuration padding token
model.config.pad_token_id = tokenizer.pad_token_id

2024-10-25 20:07:27.133451: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen2.5-0.5B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [0]:
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained(base_model_id)

def tokenize_function(examples):
    return tokenizer(
        examples["text"], 
        max_length=512, #added
        padding="max_length", 
        # padding_side="right", #added
        # add_eos_token=True,  #added      
        truncation=True)
    
tokenizer.pad_token = tokenizer.eos_token    

# tokenized_datasets = dataset.map(tokenize_function, batched=True)
small_train_dataset = sampled_train_dataset.map(tokenize_function) #, batched=True)
small_eval_dataset = sampled_test_dataset.map(tokenize_function) #, batched=True)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [0]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [0]:
# Model training

from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
from datetime import datetime

training_args = TrainingArguments(
    report_to="mlflow",
    run_name=f"{base_model_id}-{datetime.now().strftime('%Y-%m-%d-%H-%M-%s')}",
    output_dir='/local_disk0/llm_outputs', 
    per_device_train_batch_size=16, # increase to 4, 8, etc., until you encounter an OutOfMemory (OOM) error
    gradient_accumulation_steps=4, # was 4
    gradient_checkpointing=True, # was True    
    eval_strategy="steps",
    max_steps=500,
    warmup_steps=5,
    save_strategy="steps",
    save_steps=100,
    logging_strategy="steps",
    logging_steps=100,
    logging_dir='/local_disk0/llm_outputs/logs', #output_dir_logs, # Directory for training logs
    eval_steps=100,  # evaluating at every step slows down a lot the training!
    save_total_limit=2, # Reduce the total number of saved checkpoints
    load_best_model_at_end=True,
    # metric_for_best_model="eval_loss",
    # greater_is_better=False,  
    #       
    )

# Create the early stopping callback
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,  # Number of evaluation calls to wait if no improvement
    early_stopping_threshold=0.01,  # Minimum change to qualify as an improvement
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback],  # Add the early stopping callback
)

trainer.train()

max_steps is given, it will override any value given in num_train_epochs
/databricks/python/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
100,1.535000,0.953736,0.563000


/databricks/python/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/databricks/python/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/databricks/python/lib/python3.11/site-packages/torch/utils/checkp

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
# Load the base model for evaluation
base_model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)


# Set up the trainer to evaluate the base model
base_trainer = Trainer(
    model=base_model,
    args=training_args,  # Use the same training arguments
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics
)

# Evaluate the base model
base_results = base_trainer.evaluate()
print("Base Model Performance: ", base_results)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
max_steps is given, it will override any value given in num_train_epochs


Base Model Performance:  {'eval_loss': 1.613896131515503, 'eval_model_preparation_time': 0.0024, 'eval_accuracy': 0.228, 'eval_runtime': 28.0045, 'eval_samples_per_second': 35.708, 'eval_steps_per_second': 4.464}


In [0]:
# Evaluate the FT model
fine_tuned_results = trainer.evaluate()
print("Fine-Tuned Model Performance: ", fine_tuned_results)

Fine-Tuned Model Performance:  {'eval_loss': 0.8271757960319519, 'eval_accuracy': 0.654, 'eval_runtime': 26.9015, 'eval_samples_per_second': 37.173, 'eval_steps_per_second': 4.647, 'epoch': 1.6}


In [0]:
import torch
# Define a function to get predictions
# def get_prediction(text, model, tokenizer):
#     inputs = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True)
#     outputs = model(**inputs)
#     logits = outputs.logits
#     prediction = logits.argmax(-1).item()
#     return prediction

def get_prediction(text, model, tokenizer, device):
    # Move the model to the specified device
    model.to(device)
    
    # Tokenize the text and move the tensors to the device
    inputs = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True).to(device)
    
    # Set the model to evaluation mode and run inference
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        prediction = logits.argmax(-1).item()    
    return prediction

# Set the device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

sample_index = 102
sample_text = dataset["test"][sample_index]['text']
sample_label = dataset["test"][sample_index]['label']

base_prediction = get_prediction(sample_text, base_model, tokenizer, device)
fine_tuned_prediction = get_prediction(sample_text, model, tokenizer, device)

print(f"Sample Review: {sample_text}")
print(f"Base Model Prediction: {base_prediction}")
print(f"Fine-Tuned Model Prediction: {fine_tuned_prediction}")
print(f"Real label: {sample_label}")

Sample Review: We go here quite often for lunch. The food is excellent. Service is friendly, but spotty. If your in a hurry for lunch, I would advise you call ahead. I think perhaps the time it takes is because food is made to order. A great thing for the quality and taste but not so much if you have a limited amount of time. The lunch specials are a fantastic deal, and no matter what else I get , I get the spinach salad. It's scrumptious. Definitely recommend this place. Dinner I have had a couple times and the food was good, but the lunch deals are awesome!
Base Model Prediction: 0
Fine-Tuned Model Prediction: 3
Real label: 3


In [0]:
# NOW USE PEFT + LORA

In [0]:
# Loading the Base model
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, AutoTokenizer, EarlyStoppingCallback
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

base_model_id = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

# Set the padding token to be the same as eos token (if the model does not have a padding token)
tokenizer.pad_token = tokenizer.eos_token

# Load the model and make sure padding token is set in its configuration
# model = AutoModelForSequenceClassification.from_pretrained(base_model_id, num_labels=5)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, # Load the model with 4-bit quantization    
    bnb_4bit_use_double_quant=True, # Use double quantization    
    bnb_4bit_quant_type="nf4", # Use 4-bit Normal Float for storing the base model weights in GPU memory    
    bnb_4bit_compute_dtype=torch.bfloat16, # De-quantize the weights to 16-bit (Brain) float before the forward/backward pass
)
    
model = AutoModelForSequenceClassification.from_pretrained(
    base_model_id,
    quantization_config=quantization_config,
    trust_remote_code=True,
    num_labels=5
)

# Set the model configuration padding token
model.config.pad_token_id = tokenizer.pad_token_id

2024-10-25 21:07:53.325059: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen2.5-0.5B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [0]:
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained(base_model_id)

def tokenize_function(examples):
    return tokenizer(
        examples["text"], 
        max_length=512, #added
        padding="max_length", 
        # padding_side="right", #added
        # add_eos_token=True,  #added      
        truncation=True)
    
tokenizer.pad_token = tokenizer.eos_token    

# tokenized_datasets = dataset.map(tokenize_function, batched=True)
small_train_dataset = sampled_train_dataset.map(tokenize_function) #, batched=True)
small_eval_dataset = sampled_test_dataset.map(tokenize_function) #, batched=True)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [0]:
# def format_dataset(data_point):
#     prompt = f"""###SYSTEM: Based on INPUT title generate the prompt for generative model

# ###INPUT: {data_point['act']}

# ###PROMPT: {data_point['prompt']}
# """
#     tokens = tokenizer(prompt,
#         truncation=True,
#         max_length=256,
#         padding="max_length",)
#     tokens["labels"] = tokens['input_ids'].copy()
#     return tokens

In [0]:
print(small_train_dataset)

Dataset({
    features: ['label', 'text', 'input_ids', 'attention_mask'],
    num_rows: 20000
})


In [0]:
# from peft import LoraConfig, get_peft_model

# # Define LoRA configuration
# lora_config = LoraConfig(
#     r=8,                     # Low-rank dimension
#     lora_alpha=32,           # Scaling factor for LoRA
#     target_modules=["q_proj", "v_proj"],  # Target layers to apply LoRA (e.g., query and value projection layers)
#     lora_dropout=0.1,        # Dropout for LoRA layers
#     bias="none"              # Bias strategy ("none", "all", or "lora_only")
# )

# # Create a PEFT model using the base model and LoRA configuration
# peft_model = get_peft_model(model, lora_config)
# peft_model.print_trainable_parameters()

In [0]:
from peft import LoraConfig, get_peft_model

# Enabling gradient checkpointing, to make the training further efficient
model.gradient_checkpointing_enable()

# Set up the model for quantization-aware training e.g. casting layers, parameter freezing, etc.
model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    # task_type="CAUSAL_LM",
    # This is the rank of the decomposed matrices A and B to be learned during fine-tuning. A smaller number will save more GPU memory but might result in worse performance.
    r=16,
    # This is the coefficient for the learned ΔW factor, so the larger number will typically result in a larger behavior change after fine-tuning.
    lora_alpha=32,
    # Drop out ratio for the layers in LoRA adaptors A and B.
    lora_dropout=0.1,
    # We fine-tune all linear layers in the model. It might sound a bit large, but the trainable adapter size is still usually only 1 to 5% of the whole model.
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    # Bias parameters to train. 'none' is recommended to keep the original model performing equally when turning off the adapter.
    bias="none",
)

peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

trainable params: 8,798,208 || all params: 502,835,456 || trainable%: 1.7497


In [0]:
# import torch
# from torch.nn import DataParallel

# peft_model = DataParallel(peft_model)

In [0]:
# import numpy as np
# import evaluate

# metric = evaluate.load("accuracy")

# # before PEFT LORA
# # def compute_metrics(eval_pred):
# #     logits, labels = eval_pred
# #     predictions = np.argmax(logits, axis=-1)
# #     return metric.compute(predictions=predictions, references=labels)

# # def compute_metrics(eval_pred):
# #     predictions, labels = eval_pred
# #     predictions = np.argmax(predictions, axis=-1)
# #     results = metric.compute(predictions=predictions, references=labels)
    
# #     # Add the eval_ prefix to the metric name
# #     return {"eval_accuracy": results["accuracy"]}  


# # def compute_metrics(eval_pred):
# #     predictions, labels = eval_pred
# #     predictions = np.argmax(predictions, axis=-1)
# #     results = metric.compute(predictions=predictions, references=labels)
# #     print("Computing metrics:", {"eval_accuracy": results["accuracy"]})  # Debug print
# #     return {"eval_accuracy": results["accuracy"]}    

# def compute_metrics(eval_pred):
#     # Handle different possible output formats
#     if isinstance(eval_pred, tuple) and len(eval_pred) == 2:
#         if hasattr(eval_pred[0], 'logits'):
#             logits = eval_pred[0].logits
#         else:
#             logits = eval_pred[0]
#         labels = eval_pred[1]
#     else:
#         logits = eval_pred.predictions
#         labels = eval_pred.label_ids
    
#     predictions = np.argmax(logits, axis=-1)
#     results = metric.compute(predictions=predictions, references=labels)
#     return {"eval_accuracy": results["accuracy"]}     

In [0]:
# # Model training

# from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
# from datetime import datetime

# training_args = TrainingArguments(
#     report_to="mlflow",
#     run_name=f"{base_model_id}-{datetime.now().strftime('%Y-%m-%d-%H-%M-%s')}",
#     output_dir='/local_disk0/llm_outputs', 
#     per_device_train_batch_size=16, # increase to 4, 8, etc., until you encounter an OutOfMemory (OOM) error
#     per_device_eval_batch_size=8,  # Added explicit eval batch size
#     gradient_accumulation_steps=4, # was 4
#     gradient_checkpointing=True, # was True   
#     optim="paged_adamw_8bit",
#     bf16=True,     
#     eval_strategy="steps",
#     max_steps=500,
#     warmup_steps=5,
#     save_strategy="steps",
#     save_steps=3,
#     logging_strategy="steps",
#     logging_steps=3,
#     logging_dir='/local_disk0/llm_outputs/logs', #output_dir_logs, # Directory for training logs
#     eval_steps=3,  # evaluating at every step slows down a lot the training!
#     save_total_limit=2, # Reduce the total number of saved checkpoints
#     load_best_model_at_end=True,
#     # metric_for_best_model="eval_loss",
#     # greater_is_better=False,    
#     # metric_for_best_model="eval_accuracy",  # Change to match your metric
#     # greater_is_better=True,  # For accuracy, higher is better       
#     ddp_find_unused_parameters=False,  # Added this parameter 
#     gradient_checkpointing_kwargs={"use_reentrant": False},  # Added this parameter
#     label_names=['label', 'input_ids', 'attention_mask']
#     )

# # Create the early stopping callback
# early_stopping_callback = EarlyStoppingCallback(
#     early_stopping_patience=3,  # Number of evaluation calls to wait if no improvement
#     early_stopping_threshold=0.01,  # Minimum change to qualify as an improvement
# )

# # trainer = Trainer(
# #     model=peft_model,
# #     args=training_args,
# #     train_dataset=small_train_dataset,
# #     eval_dataset=small_eval_dataset,
# #     compute_metrics=compute_metrics,
# #     callbacks=[early_stopping_callback],
# # )

# # First, create a custom trainer that handles PEFT outputs
# class PeftTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):
#         labels = inputs.pop("labels")
#         outputs = model(**inputs)
#         logits = outputs.logits
#         loss = self.label_smoother(outputs, labels) if self.label_smoother is not None else outputs.loss
#         return (loss, {'logits': logits}) if return_outputs else loss

# metric = evaluate.load("accuracy")

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     results = metric.compute(predictions=predictions, references=labels)
#     return {"eval_accuracy": results["accuracy"]}

# # Use PeftTrainer instead of regular Trainer
# trainer = PeftTrainer(
#     model=peft_model,
#     args=training_args,
#     train_dataset=small_train_dataset,
#     eval_dataset=small_eval_dataset,
#     compute_metrics=compute_metrics,
#     callbacks=[early_stopping_callback],
# )

# trainer.train()

In [0]:
import numpy as np
import evaluate
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback, EvalPrediction
from datetime import datetime
import time
import torch

class PeftTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        
        # Forward pass
        outputs = model(**inputs)
        
        # Get logits and loss
        if isinstance(outputs, dict):
            logits = outputs.get("logits")
            loss = outputs.get("loss")
        else:
            logits = outputs.logits
            loss = outputs.loss
            
        # If loss is not computed by the model, compute it
        if loss is None and labels is not None:
            if self.label_smoother is not None:
                loss = self.label_smoother(outputs, labels)
            else:
                loss = torch.nn.functional.cross_entropy(
                    logits.view(-1, logits.size(-1)),
                    labels.view(-1)
                )
        
        # Make sure loss is a scalar tensor
        if isinstance(loss, dict):
            loss = sum(loss.values())

        return (loss, outputs) if return_outputs else loss

    def evaluation_loop(self, dataloader, description, prediction_loss_only=None, ignore_keys=None, metric_key_prefix="eval"):
        eval_output = super().evaluation_loop(
            dataloader, description, prediction_loss_only, ignore_keys, metric_key_prefix
        )
        
        # Calculate and add the loss metric manually
        if "eval_loss" not in eval_output.metrics and eval_output.predictions is not None:
            losses = []
            for inputs in dataloader:
                with torch.no_grad():
                    inputs = self._prepare_inputs(inputs)
                    loss = self.compute_loss(self.model, inputs)
                    losses.append(loss.item())
            eval_loss = np.mean(losses)
            eval_output.metrics["eval_loss"] = eval_loss
        
        return eval_output

# Setup training arguments with unique identifiers
unique_timestamp = f"{datetime.now().strftime('%Y-%m-%d-%H-%M')}-{int(time.time())}"
training_args = TrainingArguments(
    report_to="mlflow",
    run_name=f"{base_model_id}-{unique_timestamp}",
    output_dir=f'/local_disk0/llm_outputs/{unique_timestamp}',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
    bf16=True,
    
    max_steps=300,
    
    # Evaluation settings
    evaluation_strategy="steps",
    eval_steps=50,
    
    # Model saving settings
    save_strategy="steps",
    save_steps=50,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,        
    
    logging_strategy="steps",
    logging_steps=50,
    logging_dir=f'/local_disk0/llm_outputs/logs/{unique_timestamp}',
    ddp_find_unused_parameters=False,
    gradient_checkpointing_kwargs={"use_reentrant": False},
)

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred: EvalPrediction):
    print("Computing metrics...")  # Debug print
    if isinstance(eval_pred.predictions, tuple):
        logits = eval_pred.predictions[0]
    else:
        logits = eval_pred.predictions
    labels = eval_pred.label_ids
    
    predictions = np.argmax(logits, axis=-1)
    results = metric.compute(predictions=predictions, references=labels)
    print(f"Computed results: {results}")  # Debug print
    
    return {
        "eval_accuracy": float(results["accuracy"])  # Ensure it's a float
    }

# Create the early stopping callback
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,  # Number of evaluation calls to wait if no improvement
    early_stopping_threshold=0.01,  # Minimum change to qualify as an improvement
)

# Initialize trainer
trainer = PeftTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback],
)

# Optional: verify setup
print("Running initial evaluation...")
initial_metrics = trainer.evaluate()
print(f"Initial metrics: {initial_metrics}")

# Start training
trainer.train()


/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


Running initial evaluation...
[2024-10-25 21:10:35,969] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Initial metrics: {'eval_model_preparation_time': 0.0057, 'eval_loss': 8.510500732421875, 'eval_runtime': 44.952, 'eval_samples_per_second': 22.246, 'eval_steps_per_second': 2.781}


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss,Model Preparation Time
50,2.549000,1.213401,0.005700
100,1.181100,1.023966,0.005700
150,1.033800,0.896523,0.005700
200,0.919200,0.874798,0.005700
250,0.910100,0.827627,0.005700
300,0.859600,0.823757,0.005700


/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.11/site-packages/peft/utils/save_and_load.py:202: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.11/site-packages/peft/utils/save_and_load.py:202: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.11/site-packages/peft/utils/save_and_load.py:202: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/local_disk0/.ephemeral_n

TrainOutput(global_step=300, training_loss=1.2421293512980143, metrics={'train_runtime': 1126.8605, 'train_samples_per_second': 17.038, 'train_steps_per_second': 0.266, 'total_flos': 2.1628892661940224e+16, 'train_loss': 1.2421293512980143, 'epoch': 0.96})

In [0]:
# Load the base model for evaluation
base_model = AutoModelForSequenceClassification.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct", num_labels=5)

# Set up the trainer to evaluate the base model
base_trainer = Trainer(
    model=base_model,
    args=training_args,  # Use the same training arguments
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics
)

# Evaluate the base model
base_results = base_trainer.evaluate()
print("Base Model Performance: ", base_results)

# Evaluate the FT model
fine_tuned_results = trainer.evaluate()
print("Fine-Tuned Model Performance: ", fine_tuned_results)

Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen2.5-0.5B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
File <command-2284551959431322>, line 5
      2 base_model = AutoModelForSequenceClassification.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct", num_labels=5)
      4 # Set up the trainer to evaluate the base model
----> 5 base_trainer = Trainer(
      6     model=base_model,
      7     args=training_args,  # Use the same training arguments
      8     eval_dataset=small_eval_dataset,
      9     compute_metrics=compute_metrics
     10 )

File /local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.11/site-packages/transformers/trainer.py:404, in Trainer.__init__(self, model, args, data_collator, train_dataset, eval_dataset, tokenizer, model_init, compute_metrics, callbacks, optimizers, preprocess_logits_for_metrics)
    402 self.args = args
    403 # Seed must be set before instantiating the model when using model
--> 404 enable

In [0]:
import torch
def get_prediction(text, model, tokenizer, device):
    # Move the model to the specified device
    model.to(device)
    
    # Tokenize the text and move the tensors to the device
    inputs = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True).to(device)
    
    # Set the model to evaluation mode and run inference
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        prediction = logits.argmax(-1).item()    
    return prediction

# Set the device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

sample_index = 102
sample_text = dataset["test"][sample_index]['text']
sample_label = dataset["test"][sample_index]['label']

base_prediction = get_prediction(sample_text, base_model, tokenizer, device)
fine_tuned_prediction = get_prediction(sample_text, model, tokenizer, device)

print(f"Sample Review: {sample_text}")
print(f"Base Model Prediction: {base_prediction}")
print(f"Fine-Tuned Model Prediction: {fine_tuned_prediction}")
print(f"Real label: {sample_label}")